In [ ]:
%load_ext autoreload
%autoreload 2
import torch
from torch import optim,nn
from torch.autograd import Variable
import numpy as np
import time
from models.data_loader import DataLoader

from models.retain_copy import RETAIN

In [ ]:
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=0

In [ ]:
# hyperparameters
epochs = 30
batch_size = 50
max_seq_length = 300
min_seq_length = 5
num_classes = 268
emb_size = 128
hid_size = 128
lr = 0.001
cuda_flag = True

# data loader
D = DataLoader(batch_size=batch_size,
   data_dir='data/batches/',
    mode='train', max_seq_length=max_seq_length, min_seq_length=min_seq_length)

# import model and optimization settings
# model = RETAIN(128,128,268,False)

model = RETAIN(emb_size,hid_size,num_classes,cuda_flag)
model.release = True

criterion = nn.NLLLoss()
cnt = 0
if cuda_flag:
    model.cuda()
    criterion.cuda()

In [ ]:
# train model
lr_list = [0.001, 0.0003, 0.0001, 0.00003, 0.00001, 0.000003, 0.000001]
lr_counter = 0
lr = lr_list[lr_counter]
opt = optim.Adam(model.parameters(), lr=lr)
loss_list = []
loss_mean = 0.0
file_cnt = 0
cnt = 0
loss_dict = dict()

In [ ]:
len_train = len(D.train_list)
while file_cnt<(epochs*len(D.train_list)):
    idx = file_cnt%(len_train)
    file = D.train_list[idx]
    if file not in loss_dict:
        loss_dict[file] = []
    print("Epoch %d [%d, %d/%d] - opening file %s" %(((file_cnt+1)/len_train), file_cnt, idx, len_train, D.train_list[idx]))
    file_num = int(file.split('_')[1].split('.')[0])
    D.batch_size = int(40000/file_num)
    D.load_batch_file(file)
    loss_list = []
    for i in range(D.batch_count):
        cnt+=1
#         input_list, targets = D.get_batch()
        input_list, targets, dates = D.get_batch()
#         dates = Variable(torch.Tensor(dates), requires_grad=False)
#         if cuda_flag:
#             dates = dates.cuda()
        start = time.time()
        inputs = model.list_to_tensor(input_list)
        outputs = model(inputs)
        outputs = torch.log(outputs)
        targets = Variable(torch.LongTensor(targets)[:,-1]) # to only use last of each sequence
        if cuda_flag:
            targets = targets.cuda()
        loss = criterion(outputs.view(-1,num_classes),targets)
        loss_list.append(loss.data[0])
        if cnt%10==0:
            print('[%d] %1.3f' %(cnt,loss.data[0]))
        if cnt%500==0:
            print("Saving model at %dth step" %cnt)
            torch.save(model.state_dict(),'data/saved_weights/retain_copy_%d.pth'%(cnt))
            # create CPU version
            model2 = RETAIN(emb_size,hid_size,num_classes,False)
            if cuda_flag:
                model.cpu()
            model2.load_state_dict(model.state_dict())
            torch.save(model2.state_dict(),'data/saved_weights/retain_copy_%d_cpu.pth'%(cnt))
            del model2
            if cuda_flag:
                model.cuda()
            print("Saving at %dth step"%cnt)
        # manual loss changes
        if cnt==100:
            lr_counter+=1
            lr = lr_list[lr_counter]
            opt = optim.Adam(model.parameters(),lr=lr)
        if cnt==500:
            lr_counter+=1
            lr = lr_list[lr_counter]
            opt = optim.Adam(model.parameters(),lr=lr)        
        if loss.data[0]>10:
            import sys
            sys.exit()
#             print(loss.data[0])
        loss.backward()
        opt.step()
    print("Loss: %1.3f" %np.mean(loss_list))
    loss_dict[file].append(loss.data[0])
    file_cnt+=1

In [ ]:
# load new model
model = torch.load('/home/mjc/github/EHRVis/data/saved_weights/retain_bi_14500.pth')
model.train()
file_cnt = 4916
cnt = 14500

In [ ]:
# lower learning rate
lr_counter+=1
lr = lr_list[lr_counter]
opt = optim.Adam(model.parameters(), lr=lr)